In [ ]:

import torch 
import torch.nn as nn
import numpy as np
import pandas as pd 
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
  
raw_data = pd.read_csv('data/targetfirm_prediction_dataset_small.csv')
raw_data = raw_data.fillna(0)
data = np.array(raw_data.values)
data = data[:,1:]

# print(data)

# # GVKEY Labels
labels = data[:,0]
print(labels)
# print(np.unique(labels, return_index=True, return_counts=True, axis=0))
_,ind1,inv1,cou1 = np.unique(labels, return_index=True, return_inverse=True, return_counts=True)
print(ind1)
print(inv1)
print(cou1)
# Index of the last occurrence of the GVKEY
print((ind1+cou1-1))
target_indices = ind1+cou1-1
print("Length of labels is ", len(target_indices))


data_tensor = torch.FloatTensor(data)
# print(data_tensor)

print("*"*100)
def prepare_data_and_split(source_data, window_size, target_indices):
    returndata = [ ]
    print(len(source_data))
    print(len(source_data[0]))
    print(source_data[0])
    print("*"*100)
    x_data = []
    y_data = []
    #     print(window_size)
#     print(len(target_indices))
    for i in target_indices:
#         print("Processing --", i)
        current_index = i
        previous_index = current_index - 1
        count = 0 
#         print("Year ", source_data[:,1][current_index], " and year ", source_data[:,1][previous_index])
        while(source_data[:,1][current_index] > source_data[:,1][previous_index] and count < window_size):
            current_index-=1
            previous_index = current_index - 1
            count+=1
#         print(current_index, i)
        if(current_index == i):
#             print("Skipping this")
            continue
#         else:
#             print("Processing for ", count, " years--------------")
#         print("Processing for years start to end is ", source_data[:,1][current_index], source_data[:,1][i])
#         print("Which is ", source_data[current_index:i,3:17])
#         print("Target is ", source_data[i, 2])
        x_data.append(source_data[current_index:i,3:17])
        y_data.append(source_data[i,2])
        returndata.append((source_data[current_index:i,3:17], source_data[i,2]))
    print("Total returndata length is ", len(returndata))
    print(len(returndata[0]))
    test_size = int(np.round(0.3 * len(returndata)))
    print("Test size is ", test_size)
    train = returndata[:-test_size]
    x_train = x_data[:-test_size]
    y_train = y_data[:-test_size]
    test = returndata[-test_size:]
    x_test = x_data[-test_size:]
    y_test = y_data[-test_size:]
    return train, test, x_train, y_train, x_test, y_test


print("*"*100)
year_window = 5 # Years
train, test, x_train, y_train, x_test, y_test = prepare_data_and_split(data_tensor,year_window,target_indices)
print("*"*100)

# print(train)
print(len(train))
print(len(test))
print(len(x_train))
print(len(y_train))
print(len(x_train[0]))
print(len(y_train))
print(len(x_test))
print(len(y_test))
# test.head()


input_size = 14
hidden_size = 100 
num_layers = 2 
output_size = 1 
num_epochs = 10 
learning_rate = 0.05 

# x_train = torch.from_numpy(np.asarray(x_train)).type(torch.Tensor)
# x_test = torch.from_numpy(np.asarray(x_test)).type(torch.Tensor)
# y_train = torch.from_numpy(np.asarray(y_train, dtype=np.float32)).type(torch.Tensor)
# y_test = torch.from_numpy(np.asarray(y_test, dtype=np.float32)).type(torch.Tensor)


class LSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(LSTM, self).__init__()
        self.hidden_size = hidden_size 
        self.input_size = input_size
        self.num_layers = num_layers
    
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers)
        self.fc = nn.Linear(hidden_size, output_size)
        self.h_cell = (torch.zeros(self.num_layers,1, self.hidden_size),
                       torch.zeros(self.num_layers,1, self.hidden_size))
        
    def forward(self,x): 
        out, self.h_cell = self.lstm(x.view(len(x),1,-1),self.h_cell)
        output = self.fc(out.view(len(x),-1))
        return output[-1]
       
# class GRU(nn.Module):
#     def __init__(self, input_size, hidden_size ,num_layers, output_size):
#         super(GRU, self).__init__()
#         self.hidden_size = hidden_size 
#         self.input_size = input_size
#         self.num_layers = num_layers
        
#         self.gru = nn.GRU(input_size, hidden_size, num_layers)
#         self.fc = nn.Linear(hidden_size, output_size) 
#         self.h_cell = torch.zeros(self.num_layers,1, self.hidden_size)
        
#     def forward(self, x):
#         out, self.h_cell = self.gru(x.view(len(x),1,-1),self.h_cell)
#         output = self.fc(out.view(len(x),-1))
#         return output[-1]
    
lstm_model = LSTM(input_size = input_size, hidden_size = hidden_size, num_layers = num_layers, output_size = output_size)

print("Training LSTM" + f" model with {num_epochs} epochs:")

import time

hist = np.zeros(num_epochs)


start_time = time.time()


criterion = nn.MSELoss(reduction='mean')
optimizer = torch.optim.Adam(lstm_model.parameters(), learning_rate)
for i in range(num_epochs):
    for training_data, y_lstm_targets in train:

        y_train_pred = lstm_model(training_data)
        loss = criterion(y_train_pred, y_lstm_targets)
        print("Epoch ", i, "MSE: ", loss.item())
        hist[i] = loss.item()

        optimizer.zero_grad()
        loss.backward(retain_graph=True)
        optimizer.step()

end_time = time.time()
training_time = end_time - start_time
print("Time taken for the training is ", training_time)
